# Metric Design

Objective:
Define clear, interpretable metrics to evaluate the success of a new
product recommendation feature.

This notebook focuses on:
- Choosing the right success metrics
- Avoiding misleading measurements
- Establishing baseline values before experimentation


### Product Context

We analyze an e-commerce platform where users:
- View products
- Optionally add products to cart
- Complete purchases

A new recommendation feature is proposed to improve user conversion.
Before running an A/B test, we must define how success will be measured.


### Business Goal

Increase the likelihood that users who view products complete a purchase,
without negatively impacting revenue-related metrics.


In [1]:
import pandas as pd
from pathlib import Path

BASE_PATH = Path("/Users/sabarish/Desktop/Project_1/eCommerce behavior")

oct_df = pd.read_csv(BASE_PATH / "2019-Oct.csv", nrows=500_000)
nov_df = pd.read_csv(BASE_PATH / "2019-Nov.csv", nrows=500_000)

events = pd.concat([oct_df, nov_df], ignore_index=True)

events["event_time"] = pd.to_datetime(events["event_time"])


### Metric Framework

We categorize metrics into three groups:

1. Primary Metric  
   - Determines ship / no-ship decision

2. Secondary Metrics  
   - Help explain changes in user behavior

3. Guardrail Metrics  
   - Ensure no negative business impact


### Primary Metric: Conversion Rate (View → Purchase)

Definition:

Number of users who make at least one purchase
------------------------------------------------
Number of users who view at least one product

Rationale:
- Directly reflects the business objective
- Cart is optional in this platform
- Avoids misleading intermediate steps


In [2]:
user_funnel = (
    events
    .pivot_table(
        index="user_id",
        columns="event_type",
        values="event_time",
        aggfunc="min"
    )
    .reset_index()
)

user_funnel = user_funnel.rename(
    columns={
        "view": "view_time",
        "cart": "cart_time",
        "purchase": "purchase_time"
    }
)

user_funnel.head()


event_type,user_id,cart_time,purchase_time,view_time
0,244951053,NaT,NaT,2019-10-01 08:47:35+00:00
1,274969076,NaT,NaT,2019-11-01 06:18:48+00:00
2,275256741,NaT,NaT,2019-11-01 02:23:03+00:00
3,295643776,NaT,NaT,2019-11-01 03:12:38+00:00
4,296465302,NaT,NaT,2019-11-01 04:10:21+00:00


In [3]:
view_users = user_funnel["view_time"].notna().sum()
purchase_users = user_funnel["purchase_time"].notna().sum()

conversion_rate = purchase_users / view_users
conversion_rate


np.float64(0.08210740847084523)

### Secondary Metrics

1. Add-to-Cart Rate (View → Cart)
   - Measures engagement and intent

2. Average Order Value (AOV)
   - Measures purchase size


In [4]:
cart_users = user_funnel["cart_time"].notna().sum()
add_to_cart_rate = cart_users / view_users

add_to_cart_rate


np.float64(0.05001097843160437)

In [5]:
purchase_events = events[events["event_type"] == "purchase"]

aov = purchase_events["price"].mean()
aov


np.float64(310.71078230765255)

### Guardrail Metrics

Guardrails ensure that improvements in conversion do not come at the
expense of revenue quality.

Selected guardrail:
- Revenue Per User (RPU)


In [6]:
revenue_per_user = (
    purchase_events
    .groupby("user_id")["price"]
    .sum()
    .mean()
)

revenue_per_user


np.float64(412.3138898793198)

### Baseline Metrics Summary

- Conversion Rate (View → Purchase): baseline established
- Add-to-Cart Rate: baseline established
- Average Order Value (AOV): baseline established
- Revenue Per User (RPU): baseline established

These metrics will be used as reference points when evaluating
the A/B experiment results.


### Why This Metric Design Is Appropriate

- Metrics are aligned with the business objective
- Conversion is measured directly from view to purchase
- Optional cart behavior is handled correctly
- Guardrails protect against unintended revenue degradation

This metric design ensures clear, defensible, and actionable
product decisions.
